# Setup
---

In [1]:
#@title Install/Upgrade Relevant Packages
!pip install --upgrade -q google-genai
!pip install -q plotly
!pip install -q kaleido
!pip install -q weasyprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 5.6 MB/s eta 0:00:00


In [2]:
#@title Imports
import os

from google import genai
from google.genai import types
import base64

import pandas as pd
import numpy as np
import math

import plotly.graph_objects as go
import plotly.io as pio

import json

import mimetypes
from urllib.parse import urlparse

import typing
import copy

import kaleido

import IPython.display
from IPython.display import HTML, Markdown, display, clear_output
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

import markdown
from weasyprint import HTML
from google.colab import files

In [3]:
#@title If in Colab (non-Enterprise), Run This Cell to Authenticate
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [4]:
#@title Set Bracket Theme, # of Items, and Other Parameters
BRACKET_THEME = "Ben & Jerry's Ice Cream Flavors" #@param{type:"string"}

NUM_ITEMS = 64 #@param{type:"raw"} [16, 32, 64]

# Eventually add functionality for different region #s (only allow 4 for now)
NUM_REGIONS = 4 #@param {type: "raw"} [4]

GROUNDING_SOURCE = "Provided Material" #@param{type:"string"} ["None", "Google Search", "Provided Material"]

PROVIDED_MATERIAL_FILE_URI = "gs://genai_public/brackets/grounding_sources/2024_ben_and_jerrys_flavors.png" #@param{type:"string"}

BRACKET_ADVANCEMENT_SPECIAL_CRITERIA = "cookie dough is the best" #@param{type:"string"}

NUM_ROUNDS = math.ceil(math.log2(NUM_ITEMS))

RANKS = [str(i) for i in range(1, NUM_ITEMS + 1)]

NUM_SEEDS_PER_REGION = NUM_ITEMS // NUM_REGIONS

REGION_NUMS = [str(i) for i in range(1, NUM_REGIONS + 1)]

SEEDS = [str(i) for i in range(1, NUM_SEEDS_PER_REGION + 1)]

BRACKET_TITLE = f"{BRACKET_THEME} Bracket from Gemini"

BRACKET_FOOTER = (f"{BRACKET_THEME} and rankings for bracket generated by "
  "Gemini, with advancement determined by Gemini with preference for item seed"
  f'''{(" and special criteria of '" + BRACKET_ADVANCEMENT_SPECIAL_CRITERIA +
    "'") if BRACKET_ADVANCEMENT_SPECIAL_CRITERIA != "" else ""}.'''
  "<br>See more at goo.gle/gemini-bracket-code."
  )

BRACKET_DESCRIPTION = f'''{BRACKET_TITLE}:
- {NUM_ITEMS} {BRACKET_THEME} divided into {NUM_REGIONS} regions
- {NUM_SEEDS_PER_REGION} seeds per region with regions {REGION_NUMS}
- seeds {SEEDS} in each region
- {NUM_ROUNDS} rounds
- grounding source: {GROUNDING_SOURCE}
{(('- provided grounding material: ' + PROVIDED_MATERIAL_FILE_URI)
  if GROUNDING_SOURCE == 'Provided Material' else '')}

{BRACKET_FOOTER}
'''

print(BRACKET_DESCRIPTION)

Ben & Jerry's Ice Cream Flavors Bracket from Gemini:
- 64 Ben & Jerry's Ice Cream Flavors divided into 4 regions
- 16 seeds per region with regions ['1', '2', '3', '4']
- seeds ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'] in each region
- 6 rounds
- grounding source: Provided Material
- provided grounding material: gs://genai_public/brackets/grounding_sources/2024_ben_and_jerrys_flavors.png

Ben & Jerry's Ice Cream Flavors and rankings for bracket generated by Gemini, with advancement determined by Gemini with preference for item seed and special criteria of 'cookie dough is the best'.<br>See more at goo.gle/gemini-bracket-code.



In [5]:
#@title Set Up GenAI Client and Models
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

GENAI_CLIENT = genai.Client(
  vertexai=True,
  project=PROJECT_ID,
  location=LOCATION
  )

GEMINI_MODEL_ID = "gemini-2.0-flash-001" # @param {type: "string"}

IMAGEN_MODEL_ID = "imagen-3.0-generate-002" # @param {type: "string"}

In [6]:
#@title Various Helpful Functions
# Function to wrap text after certain # of characters to use on bracket
def wrap_text(text, max_chars):
    words = text.split()
    wrapped_text = ""
    current_line = ""
    for word in words:
        if len(current_line) == 0:
            current_line = word
        elif len(current_line) + len(word) + 1 <= max_chars:
            current_line += " " + word
        else:
            wrapped_text += current_line + "<br>"
            current_line = word
    wrapped_text += current_line
    return wrapped_text

# Function to get mime type from GCS URI
def get_mime_type_from_gcs_uri(file_uri):
  """Gets the MIME type from a GCS file URI.

  Args:
    file_uri: The GCS file URI.

  Returns:
    The MIME type of the file.
  """
  parsed_url = urlparse(file_uri)
  file_path = parsed_url.path.lstrip('/')
  mime_type, _ = mimetypes.guess_type(file_path)
  return mime_type

# Function to help display images in notebook
def display_image(
    image,
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)

# Function to encode image
def get_image_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode()

In [7]:
#@title Other Clerical Stuff

# Expand max column width when displaying data frames to handle longer text
pd.set_option('display.max_colwidth', 200)

# Bracket Creation Setup
---

In [8]:
#@title Create Table with All Possible Round Names Up to 7-Round Bracket
BRACKET_ROUNDS = pd.DataFrame({
  'round_desc': list(range(7, -1, -1)),
  'round_name': ['Round of 128', 'Round of 64', 'Round of 32',
    'Round of 16', 'Quarterfinals', 'Semifinals', 'Final', 'Champion']
  })

BRACKET_ROUNDS['num_items_in_round'] = 2**BRACKET_ROUNDS['round_desc']

BRACKET_ROUNDS['num_matchups_in_round'] = np.where(
  BRACKET_ROUNDS['round_desc']== 0,
  0,
  BRACKET_ROUNDS['num_items_in_round'] // 2
  )

In [9]:
#@title Functions to Create Brackets with Various Pieces
def create_df_to_represent_bracket(num_items, round_nums_to_names_df):

    num_rounds = math.ceil(math.log2(num_items))
    # Calculate the number of slots in each round
    slots_per_round = []

    # Create a list to store all the rows for the DataFrame
    all_rows = []

    # Iterate through the rounds and generate rows for each slot
    for round_num in range(num_rounds+1):
      slots_per_round.append(int(num_items / (2**round_num)))
      for slot_num in range(slots_per_round[round_num]):
          row = {
            'round_asc': round_num + 1,
            'round_desc': num_rounds - round_num,
            'round_slot': slot_num + 1
            }
          all_rows.append(row)

    # Create dataframe from list of rows, merge in round names
    bracket_df = pd.merge(
      round_nums_to_names_df,
      pd.DataFrame(all_rows),
      on = ['round_desc'],
      how = 'right'
      )

    return bracket_df


def get_slot_seed_ordering_df(num_seeds = 16):
  closest_power_of_2 = 2**(math.ceil(math.log2(num_seeds)))

  if(closest_power_of_2 == 2):
    seeds_power2 = [1, 2]

  elif(closest_power_of_2 == 4):
    seeds_power2 = [1, 4, 3, 2]

  elif(closest_power_of_2 == 8):
    seeds_power2 = [1, 8, 4, 5, 3, 6, 2, 7]

  elif(closest_power_of_2 == 16):
    seeds_power2 = [1, 16, 8, 9, 5, 12, 4, 13, 6, 11, 3, 14, 7, 10, 2, 15]

  final_seeds_order = [(seed if seed <= num_seeds else pd.NA)
    for seed in seeds_power2]

  slot_seed_ordering_df = pd.DataFrame({
    'slot': range(1, closest_power_of_2 + 1),
    'seed': final_seeds_order
    })

  return(slot_seed_ordering_df)


def get_bracket_viz_config(num_items):
    """Returns a dictionary containing bracket configuration constants."""

    num_rounds = math.ceil(math.log2(num_items))

    config = {
        "bracket_line_color": 'black',
        "bracket_line_width": 1,
        "single_rect_width": 122,
        "single_rect_height": 22,
        "between_round_space_width": 14,
        "between_rect_space_height": 15,
        "num_rounds": num_rounds,
        "round0_rect_inflation_factor": 1.25,
        "round_name_space_vs_max_y": 12,
        "round_name_height": 30,
        "round_name_font_size": 18,
        "title_space_vs_max_round_name_y": 16,
        "title_font_size": 30,
        "footer_space_vs_min_bracket_rect_y": 16,
        "footer_font_size": 14,
        "vertical_margin_space": 15,
        "overall_width": (num_rounds + 1) * 260,
        "image_space_vs_min_y": 12,
        "image_width": 8 * (num_rounds**2) - 28,
        "image_height": 8 * (num_rounds**2) - 28
        }

    config["overall_height"] = config["overall_width"] * 8.5 / 11
    config["left_end"] = -config["overall_width"] / 2
    config["right_end"] = config["overall_width"] / 2
    config["top"] = config["overall_height"] / 2
    config["bottom"] = -config["overall_height"] / 2

    return config


def create_empty_figure_for_bracket(num_items):
    """Creates an empty Plotly bracket figure using provided configuration."""
    BrCo = get_bracket_viz_config(num_items)

    bracket_layout = go.Figure()

    axis_dict = dict(title=None, showticklabels=False, showgrid=False,
      zeroline=False, scaleratio=1)

    bracket_layout.update_layout(
      margin=dict(t=0, r=0, b=0, l=0),
      autosize=False,
      width=BrCo["overall_width"],
      height=BrCo["overall_height"],
      xaxis=axis_dict,
      yaxis=axis_dict,
      plot_bgcolor="white",
      showlegend=False,
      )

    bracket_layout.update_xaxes(range=[BrCo["left_end"], BrCo["right_end"]])
    bracket_layout.update_yaxes(range=[BrCo["bottom"], BrCo["top"]])

    return bracket_layout


def get_bracket_rect_top_y_by_slot_num(slot_num, num_items_in_round,
    round_vs_ovr_num, between_rect_space, rect_height,
    connected_matchups):

    rank = (slot_num-1) % (num_items_in_round/2) + 1

    num_spaces = np.where(connected_matchups,
      (num_items_in_round/4 + 1) - ((rank + 1) // 2) * 2,
      ((num_items_in_round/2 - (2 * rank) + 1) *
          (2 **(round_vs_ovr_num - 1)))
      )

    num_rects = np.where(connected_matchups,
      (num_items_in_round/4) - rank + 1,
      num_spaces + 0.5
      )

    rect_top_y = np.where(
      (num_items_in_round <= 2), rect_height / 2,
      (num_spaces * between_rect_space) + (num_rects * rect_height)
      )

    return(rect_top_y)


def get_bracket_visual_element_dfs_from_bracket_df(num_items, start_bracket_df):
    BrCo = get_bracket_viz_config(num_items)

    num_rounds = math.ceil(math.log2(num_items))

    bracket_df = start_bracket_df.copy()

    bracket_df['rect_width'] = (BrCo['single_rect_width'] *
      np.where(bracket_df['round_desc'] == 0,
        BrCo['round0_rect_inflation_factor'],
        1)
      )

    bracket_df['rect_height'] = (BrCo['single_rect_height'] *
      np.where(bracket_df['round_desc'] == 0,
        BrCo['round0_rect_inflation_factor'],
        1)
      )

    bracket_df['round_width_offset'] = np.where(bracket_df['round_desc'] == 0,
      0,
      ((bracket_df['round_desc'] - 1) *
        (BrCo['between_round_space_width'] + bracket_df['rect_width'])
        +
        BrCo['single_rect_width'] * BrCo['round0_rect_inflation_factor']
        )
      )

    bracket_df['round_space_height'] = (bracket_df['round_asc'] *
      (BrCo['between_rect_space_height'] + BrCo['single_rect_height']))

    bracket_df['rect_left_x'] = np.where(bracket_df['round_slot'] <=
      (bracket_df['num_items_in_round'] // 2),
      -bracket_df['round_width_offset'] - bracket_df['rect_width']/2,
      bracket_df['round_width_offset'] - bracket_df['rect_width']/2
      )

    bracket_df['rect_right_x'] = np.where(bracket_df['round_slot'] <=
      (bracket_df['num_items_in_round'] // 2),
      -bracket_df['round_width_offset'] + bracket_df['rect_width']/2,
      bracket_df['round_width_offset'] + bracket_df['rect_width']/2
      )

    bracket_df['rect_mid_x'] = (bracket_df['rect_left_x'] +
      bracket_df['rect_right_x']) / 2

    bracket_df['rect_top_y'] = get_bracket_rect_top_y_by_slot_num(
      slot_num = bracket_df['round_slot'],
      num_items_in_round = bracket_df['num_items_in_round'],
      round_vs_ovr_num = (num_rounds - bracket_df['round_desc']
        ).astype('float'),
      between_rect_space = BrCo['between_rect_space_height'],
      rect_height = bracket_df['rect_height'],
      # connected_matchups = False,
      # Only connect matchups for 1st round for now
      connected_matchups = np.where(num_rounds == bracket_df['round_desc'],
        True, False)
      )

    bracket_df['rect_bottom_y'] = (bracket_df['rect_top_y'] -
      bracket_df['rect_height'])

    bracket_df['rect_mid_y'] = (bracket_df['rect_top_y'] +
      bracket_df['rect_bottom_y']) / 2

    # Get next round info
    bracket_df['next_round_desc'] = np.where(bracket_df['round_desc'] == 0,
      pd.NA, bracket_df['round_desc'] - 1)

    # Next round slot is grouping pairs - divide current round by 2 & round up
    bracket_df['next_round_slot'] = np.where(bracket_df['round_desc'] == 0,
      pd.NA, np.ceil(bracket_df['round_slot'] / 2).astype(int))

    next_round_cols = ['round_desc', 'round_slot', 'rect_left_x',
      'rect_right_x', 'rect_mid_x', 'rect_top_y', 'rect_bottom_y', 'rect_mid_y']

    # Self-join to get next round rectangle info to help draw connectors
    bracket_df = pd.merge(
      bracket_df,
      bracket_df[next_round_cols].rename(
        columns={col: 'next_' + col for col in next_round_cols}
        ),
      on = ['next_round_desc', 'next_round_slot'],
      how = "left"
      )

    bracket_df['next_round_horiz_dir'] = np.where(
      bracket_df['round_desc'] == 0,
      'NA',
      np.where(
        bracket_df['rect_mid_x'] < bracket_df['next_rect_mid_x'],
        'right',
        np.where(
          bracket_df['rect_left_x'] > bracket_df['next_rect_left_x'],
          'left',
          'same'
          )
        )
      )

    bracket_df['rect_outgoing_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['next_round_horiz_dir'] == 'left',
        bracket_df['rect_left_x'],
        np.where(
          bracket_df['next_round_horiz_dir'] == 'right',
          bracket_df['rect_right_x'],
          # Placeholder for "same" (not clear what it should be)
          bracket_df['rect_mid_x']
          )
        )
      )

    bracket_df['next_rect_incoming_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['next_round_horiz_dir'] == 'left',
        bracket_df['next_rect_right_x'],
        np.where(
          bracket_df['next_round_horiz_dir'] == 'right',
          bracket_df['next_rect_left_x'],
          # Placeholder for "same"
          bracket_df['next_rect_mid_x']
          )
        )
      )

    bracket_df['halfway_to_next_round_x'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      (bracket_df['rect_outgoing_x'] + bracket_df['next_rect_incoming_x']) / 2
      )

    bracket_df['next_round_vert_dir'] = np.where(
      bracket_df['round_desc'] == 0,
      pd.NA,
      np.where(
        bracket_df['rect_mid_y'] < bracket_df['next_rect_mid_y'],
        'up',
        np.where(
          bracket_df['rect_top_y'] > bracket_df['next_rect_top_y'],
          'down',
          'same'
          )
        )
      )

    round_names_df = (bracket_df.
      assign(
        next_round_horiz_dir = lambda x: np.where(
          x['round_desc'] == 0,
          "NA",
          x['next_round_horiz_dir']
          )
        ).
        groupby(['round_desc', 'round_name', 'next_round_horiz_dir']).
        agg(
          round_text_left_x=('rect_left_x', 'min'),
          round_text_right_x=('rect_right_x', 'max'),
          round_rect_top_y=('rect_top_y', 'max'),
          round_rect_bottom_y=('rect_bottom_y', 'min')
          ).
        reset_index()
      )

    round_names_df['round_text_mid_x'] = (round_names_df['round_text_left_x'] +
      round_names_df['round_text_right_x']) / 2

    round_names_df['round_name_top_y'] = np.where(
      round_names_df['round_desc'] == 0,
      round_names_df['round_rect_top_y'],
      max(round_names_df['round_rect_top_y'])
      ) + BrCo['round_name_space_vs_max_y'] + BrCo['round_name_height']

    round_names_df['round_name_font_size'] = BrCo['round_name_font_size']

    bracket_and_round_names_dfs_dict = dict({"bracket_df": bracket_df,
      "round_names_df": round_names_df})

    return(bracket_and_round_names_dfs_dict)


def build_bracket_viz_from_dfs(num_items, bracket_df, round_names_df,
    bracket_title = None, bracket_footer = None,
    trim_bracket_viz_vertically = False
    ):
    BrCo = get_bracket_viz_config(num_items)

    bracket_viz = create_empty_figure_for_bracket(num_items)

    for index, row in bracket_df.iterrows():
        # Add bracket rectangles for each slot, 1 by 1
        bracket_viz.add_shape(
          dict(
            type="rect",
            xref="x",
            yref="y",
            x0=row['rect_left_x'],
            y0=row['rect_top_y'],
            x1=row['rect_right_x'],
            y1=row['rect_bottom_y'],
            line_color=BrCo['bracket_line_color'],
            line_width=BrCo['bracket_line_width']
            )
          )

        # Add connectors for all rounds except 0
        if(row['round_desc'] != 0):
          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['rect_outgoing_x'],
              y0=row['rect_mid_y'],
              x1=row['halfway_to_next_round_x'],
              y1=row['rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['halfway_to_next_round_x'],
              y0=row['rect_mid_y'],
              x1=row['halfway_to_next_round_x'],
              y1=row['next_rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

          bracket_viz.add_shape(
            dict(
              type="line",
              x0=row['halfway_to_next_round_x'],
              y0=row['next_rect_mid_y'],
              x1=row['next_rect_incoming_x'],
              y1=row['next_rect_mid_y'],
              line_color=BrCo['bracket_line_color'],
              line_width=BrCo['bracket_line_width']
              )
            )

    # Add each round name above corresponding bracket slot column(s)
    for index, row in round_names_df.iterrows():
        # Add round name above each "column" of brackets
        bracket_viz.add_annotation(
          dict(
            x=row['round_text_mid_x'],
            y=row['round_name_top_y'],
            text=row['round_name'],
            font=dict(size = row['round_name_font_size']),
            showarrow=False,
            xanchor="center",
            yanchor="top"
            )
          )

    # Add title to bracket
    if(bracket_title is not None and bracket_title != ""):
      # Take average of furthest round name text x-coordinates on either side
      title_text_mid_x = (round_names_df['round_text_left_x'].min() +
        round_names_df['round_text_right_x'].max()) / 2

      # Take max of round name text y-coordinates and add spacing + font size
      title_text_top_y = (round_names_df['round_name_top_y'].max() +
        BrCo['title_space_vs_max_round_name_y']) + BrCo['title_font_size']

      bracket_viz.add_annotation(
        dict(
          x=title_text_mid_x,
          y=title_text_top_y,
          text=bracket_title,
          font=dict(size = BrCo['title_font_size']),
          showarrow=False,
          xanchor="center",
          yanchor="top"
          )
        )

    # Add footer to bracket
    if(bracket_footer is not None and bracket_footer != ""):
      # Take average of furthest round name text x-coordinates on either side
      footer_text_mid_x = (round_names_df['round_text_left_x'].min() +
        round_names_df['round_text_right_x'].max()) / 2

      footer_text_max_length = (round_names_df['round_text_right_x'].max() -
        round_names_df['round_text_left_x'].min()) - 200

      # Take min of bracket df text y-coordinates and subtract spacing
      footer_text_top_y = (bracket_df['rect_bottom_y'].min() -
        BrCo['footer_space_vs_min_bracket_rect_y'])

      # Wrap bracket footer if very long
      bracket_footer_wrapped = wrap_text(bracket_footer, 150)

      bracket_viz.add_annotation(
        dict(
          x=footer_text_mid_x,
          y=footer_text_top_y,
          text=bracket_footer_wrapped,
          font=dict(size = BrCo['footer_font_size']),
          showarrow=False,
          xanchor="center",
          yanchor="top"
          )
        )

    return bracket_viz

def add_bracket_items_to_bracket_viz(bracket_viz, bracket_df, bracket_items_df,
    max_display_char_width = 15):

    bracket_items_df['chunk_display_text'] = (bracket_items_df.
      apply(lambda row:
        wrap_text(f"{row['display_text']}", max_display_char_width),
        axis=1
        )
      )

    bracket_items_for_display = pd.merge(
      bracket_items_df,
      bracket_df,
      on = ["round_desc", "round_slot"],
      how = "left"
      )

    bracket_items_for_display['display_text_font_size'] = (
      bracket_items_for_display['rect_width'] /
      (bracket_items_for_display['chunk_display_text'].str.count('<br>') + 1)
      ) * 0.12

    bracket_viz_with_items = copy.deepcopy(bracket_viz)

    for index, row in bracket_items_for_display.iterrows():
        bracket_viz_with_items.add_annotation(
          dict(
            x=row['rect_left_x'],
            y=row['rect_top_y'],
            text=row['chunk_display_text'],
            font=dict(size = row['display_text_font_size']),
            hovertext=row['hover_text'],
            showarrow=False,
            xanchor="left",
            yanchor="top"
            )
          )

    return bracket_viz_with_items

def build_bracket_viz(num_items, round_nums_to_names_df, bracket_title = None,
    bracket_footer = None, bracket_items_df = None, existing_bracket_viz = None,
    trim_bracket_viz_vertically = False
    ):

    BrCo = get_bracket_viz_config(num_items)

    initial_bracket_df = create_df_to_represent_bracket(num_items,
      round_nums_to_names_df)

    bracket_dfs = get_bracket_visual_element_dfs_from_bracket_df(
      num_items, initial_bracket_df)

    if(existing_bracket_viz is None or not existing_bracket_viz.layout):
      bracket_viz = build_bracket_viz_from_dfs(
          num_items,
          bracket_dfs['bracket_df'],
          bracket_dfs['round_names_df'],
          bracket_title,
          bracket_footer
          )
    else:
      bracket_viz = existing_bracket_viz

    if(bracket_items_df is not None and not bracket_items_df.empty):
      bracket_viz_with_items = add_bracket_items_to_bracket_viz(
          bracket_viz,
          bracket_dfs['bracket_df'],
          bracket_items_df
          )

    else:
      bracket_viz_with_items = copy.deepcopy(bracket_viz)

    if(trim_bracket_viz_vertically):
      # Extract annotations into df
      annotation_list = []

      for annotation in bracket_viz_with_items.layout.annotations:
        annotation_dict = annotation.to_plotly_json()
        annotation_list.append(annotation_dict)

      annotation_df = pd.DataFrame(annotation_list)

      annotation_df['font_size'] = [font['size'] for font in
        annotation_df['font']]

      annotation_df['text_num_lines'] = (annotation_df['text'].str.count('<br>')
        + 1)

      # Find top / bottom text position of each annotation
      annotation_df['top_y'] = (annotation_df['y'] +
        annotation_df['font_size'] * annotation_df['text_num_lines'] *
        np.where(annotation_df['yanchor'] == "top", 0,
          np.where(annotation_df['yanchor'] == "bottom", 1, 0.5)
          )
        )

      annotation_df['bottom_y'] = (annotation_df['y'] -
        annotation_df['font_size'] * annotation_df['text_num_lines'] *
        np.where(annotation_df['yanchor'] == "top", 1,
          np.where(annotation_df['yanchor'] == "bottom", 0, 0.5)
          )
        )

      # Find highest and lowest annotations
      highest_annotation = annotation_df['top_y'].max()
      lowest_annotation = annotation_df['bottom_y'].min()

      # Adjust y-axis range and height according to bracket text extremes
      bracket_viz_with_items.update_layout(
        yaxis_range = [lowest_annotation - BrCo["vertical_margin_space"],
          highest_annotation + BrCo["vertical_margin_space"]],
        height = highest_annotation - lowest_annotation +
          BrCo["vertical_margin_space"] * 2
        )

    # Create dict of initial brackets, bracket_dfs, and bracket_viz to return
    bracket_viz_and_dfs_dict = dict({
      "initial_bracket_df": initial_bracket_df,
      "bracket_dfs": bracket_dfs,
      "bracket_viz": bracket_viz_with_items
      })

    return bracket_viz_and_dfs_dict

In [10]:
#@title Create Initial Bracket Shell
empty_bracket_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_items_df = None,
  existing_bracket_viz = None,
  trim_bracket_viz_vertically = False
  )

empty_bracket_viz = empty_bracket_viz_and_dfs['bracket_viz']

display(empty_bracket_viz)

# Gemini-Based Ranked List Generation
---

In [11]:
#@title Functions to Get Ranked Item List & Extract Using Controlled Generation
def get_list_of_items_that_fit_bracket_theme(num_items, bracket_theme,
    grounding_source, provided_material_file_uri):

    list_generation_system_instruction = types.Part.from_text(text="""
    You are an expert rankings list creator, who can create rankings lists of
    various sizes given a specific theme.

    If Google Search access is enabled, use search results in creating the list.

    If files are provided with items and information relevant to the theme of
    the list, use information directly in the files in creating the list.

    Make sure that each item returned is real, has a unique name (no duplicates
    - add text to clarify if items are similar), and fits within the list theme.

    Rank the items based on the prevalence, popularity, quality, or otherwise
    superiority of the items (with criteria depending on the theme of the
    bracket).

    Provide a description of each item including an explanation of why it was
    ranked where it was, including all items through the end of the ranking.

    Only return the list of items in ranked order along with the description or
    explanation, no other extra information.
    """)

    list_generation_tools = ([types.Tool(google_search=types.GoogleSearch())] if
      grounding_source == "Google Search" else [])

    list_generation_config = types.GenerateContentConfig(
      temperature = 1,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [list_generation_system_instruction],
      response_modalities = ["TEXT"],
      tools = list_generation_tools
      )

    list_generation_prompt = types.Part.from_text(text=
    f"Create a ranked list of {num_items} {bracket_theme}.")

    content_generation_parts = [list_generation_prompt]

    if (grounding_source == "Provided Material" and provided_material_file_uri
        and provided_material_file_uri != ""):
        list_generation_ref_material = types.Part.from_uri(
          file_uri=provided_material_file_uri,
          mime_type=get_mime_type_from_gcs_uri(provided_material_file_uri)
          )

        content_generation_parts = (content_generation_parts +
          [list_generation_ref_material])

    list_generation_contents = [
      types.Content(
        role="user",
        parts=content_generation_parts
        )
      ]

    list_generation_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=list_generation_contents,
      config=list_generation_config
      )

    list_generation_response_text = list_generation_response.text

    # display(Markdown(list_generation_response_text))

    return(list_generation_response_text)


def get_ranked_list_from_text_response(num_items, list_generation_text_response
    ):
    ranked_list_response_schema = {
      "type": "ARRAY",
      "items": {
        "type": "OBJECT",
        "properties": {
          "item": {
            "type": "STRING"
          },
          "description": {
            "type": "STRING"
          },
          "rank": {
            "type": "STRING",
            "enum": RANKS
          },
          "short_name": {
            "type": "STRING"
          }
        },
        "required": [
          "item",
          "description",
          "rank",
          "short_name"
          ]
      }
    }

    ranked_list_parsing_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        response_modalities = ["TEXT"],
        response_mime_type = "application/json",
        response_schema = ranked_list_response_schema
        )

    ranked_list_generation_prompt = types.Part.from_text(text=f"""
    Parse the provided text containing a ranked list into a JSON object with
    {num_items} elements that adheres to the following schema:
    {ranked_list_response_schema}
    Item, ranking, and description should come directly from the text.
    short_name should be a shorter version of the item name that is 10
    characters or fewer, including spaces (can be an abbreviation). Each item,
    ranking, description, and short_name should be unique - there should no
    duplicates across the list in any field.
    Here is the text with the ranked list:
    {list_generation_text_response}
    """)

    ranked_list_generation_contents = [
      types.Content(
        role="user",
        parts=[
          ranked_list_generation_prompt
        ]
        )
      ]

    ranked_list_generation_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=ranked_list_generation_contents,
      config=ranked_list_parsing_config
      )

    ranked_list_response_json = ranked_list_generation_response.text

    return(ranked_list_response_json)

def get_ranked_df_of_bracket_theme_items(num_items, bracket_theme,
    grounding_source, provided_material_file_uri, num_retries = 10):

    for i in range(num_retries):
      # First get items list in text form
      items_list_for_bracket_text = get_list_of_items_that_fit_bracket_theme(
        num_items = NUM_ITEMS,
        bracket_theme = BRACKET_THEME,
        grounding_source = GROUNDING_SOURCE,
        provided_material_file_uri = PROVIDED_MATERIAL_FILE_URI
        )

      # Then extract ranked list in JSON form
      ranked_list_response_json = get_ranked_list_from_text_response(
          NUM_ITEMS, items_list_for_bracket_text)

      # Convert to DF
      ranked_list_df = (pd.DataFrame(
        json.loads(ranked_list_response_json)).
        astype({
          'rank':'int64'
          })
        )

      # Check that all fields have unique values across all num_items
      all_fields_unique = (ranked_list_df.nunique() == NUM_ITEMS).all()

      if all_fields_unique:
        return ranked_list_df
      else:
        print("All ranking fields not unique, trying to generate again.")

    raise RuntimeError("Failed to generate a valid ranked list of items after "
      f"{num_retries} retries.")

In [12]:
#@title Call Function to Get Ranked DF of Specific # Items from Bracket Theme

ranked_list_df = get_ranked_df_of_bracket_theme_items(
  num_items = NUM_ITEMS,
  bracket_theme = BRACKET_THEME,
  grounding_source = GROUNDING_SOURCE,
  provided_material_file_uri = PROVIDED_MATERIAL_FILE_URI
  )

display(ranked_list_df)

,item,description,rank,short_name
0,Cherry Garcia,"A classic for a reason, this flavor combines cherry ice cream with cherries and fudge flakes, offering a simple yet satisfying flavor profile that has stood the test of time.",1,Cherry Gar
1,Chocolate Chip Cookie Dough,"This beloved flavor features vanilla ice cream with gobs of chocolate chip cookie dough, a combination that appeals to a wide range of ice cream lovers with its classic mix-ins.",2,Choc Chip
2,Half Baked,"Combining chocolate and vanilla ice creams with gobs of chocolate chip cookie dough and fudge brownies, this flavor offers a textural and flavorful experience that is both indulgent and comforting.",3,Half Baked
3,The Tonight Dough,"With caramel and chocolate ice creams, chocolate cookie swirls, and gobs of chocolate chip cookie dough and peanut butter cookie dough, this flavor is a rich and complex treat that keeps you comin...",4,Tonight Dough
4,Phish Food,"This flavor blends chocolate ice cream with gooey marshmallow swirls, caramel swirls, and fudge fish, creating a unique and playful combination of textures and flavors that is both fun and delicious.",5,Phish Food
...,...,...,...,...
59,Chocolate Fudge Brownie (Non-Dairy),Chocolate non-dairy frozen dessert with fudge brownies.,60,Brownie ND
60,Colin Kaepernick's Change the Whirled,"Caramel non-dairy frozen dessert with fudge chips, graham cracker swirls, and chocolate cookie swirls.",61,Change World
61,Karamel Sutra Core (Non-Dairy),Chocolate and caramel non-dairy frozen dessert with fudge chips and a soft caramel core.,62,Sutra ND
62,Lights! Caramel! Action! (Non-Dairy),"Vanilla non-dairy frozen dessert with salted caramel swirls, graham cracker swirls, and gobs of chocolate chip cookie dough.",63,Lights! ND


# Bracket Generation Using Ranked List
---

In [13]:
#@title Bracket Generation from Ranked List
def get_bracket_from_ranked_list(bracket_theme, num_items, num_regions,
    ranked_list_df):

    ranked_list_json_records = ranked_list_df.to_json(orient = 'records')

    num_items_per_region = num_items // num_regions

    bracket_generation_system_instruction = types.Part.from_text(text="""
    You are an expert bracket creator, who can create brackets of various sizes
    (16, 32, or 64 items) from a provided ranked list of items that follow a
    specific theme.

    Brackets created should follow "usual" bracket rules like an equal number of
    items in each region, items seeded 1 to the number of items in each region,
    and matchups based on seeding (e.g. in a 64-item, 4-region bracket: 1 vs 16,
    2 vs 15, and so on). Seedings should be based on the original rankings
    provided (better ranks get lower seeds like 1, 2, 3, etc.).

    Make sure that each item in the bracket is directly from the rankings list
    and used exactly ONCE in the bracket - no items should be duplicated, and
    none should be left out. There should only be 1 of each seed # in each
    region. Short names, descriptions, and original ranks should be taken
    exactly from the provided rankings list (do not change them).

    If possible, put items into regions based on logical groupings, though this
    is not necessary since the primary goal is to get seeds right within and
    across regions. It's fine if the region names are just generic groupings
    from the big picture theme or just Region 1, 2, 3, ... .
    """)

    bracket_generation_response_schema = {
      "type": "ARRAY",
      "items": {
        "type": "OBJECT",
        "properties": {
          "item": {
            "type": "STRING"
          },
          "short_name": {
            "type": "STRING"
          },
          "description": {
            "type": "STRING"
          },
          "original_rank": {
            "type": "STRING",
            "enum": RANKS
          },
          "region_num": {
            "type": "STRING",
            "enum": REGION_NUMS
          },
          "region_name": {
            "type": "STRING"
          },
          "seed": {
            "type": "STRING",
            "enum": SEEDS
          }
        },
        "required": [
          "item",
          "short_name",
          "description",
          "original_rank",
          "region_num",
          "region_name",
          "seed"
        ]
      }
    }

    bracket_generation_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        system_instruction = [bracket_generation_system_instruction],
        response_modalities = ["TEXT"],
        response_mime_type = "application/json",
        response_schema = bracket_generation_response_schema
        )

    bracket_generation_prompt = types.Part.from_text(text=f"""
    Create a {num_items}-item bracket of {bracket_theme}, including
    {num_regions} regions with {num_items_per_region} items per region, using
    the following ranked list:
    {ranked_list_json_records}

    Return the bracket as a JSON object with {num_items} elements that adheres
    to the following schema: {bracket_generation_response_schema}
    """)

    bracket_generation_contents = [
      types.Content(
        role="user",
        parts=[
          bracket_generation_prompt
        ]
        )
      ]

    bracket_generation_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=bracket_generation_contents,
      config=bracket_generation_config
      )

    bracket_generation_response_json = bracket_generation_response.text

    return(bracket_generation_response_json)

bracket_generation_response_json = get_bracket_from_ranked_list(BRACKET_THEME,
  NUM_ITEMS, NUM_REGIONS, ranked_list_df)

display(bracket_generation_response_json)

'[\n  {\n    "item": "Cherry Garcia",\n    "short_name": "Cherry Gar",\n    "description": "A classic for a reason, this flavor combines cherry ice cream with cherries and fudge flakes, offering a simple yet satisfying flavor profile that has stood the test of time.",\n    "original_rank": "1",\n    "region_num": "1",\n    "region_name": "Fruity Flavors",\n    "seed": "1"\n  },\n  {\n    "item": "Milk & Cookies",\n    "short_name": "Milk&Cookies",\n    "description": "Vanilla ice cream with a chocolate cookie swirl, chocolate chip, and chocolate chocolate chip cookies offers a classic and comforting combination.",\n    "original_rank": "21",\n    "region_num": "1",\n    "region_name": "Fruity Flavors",\n    "seed": "16"\n  },\n  {\n    "item": "Coffee Toffee Bar Crunch",\n    "short_name": "Coffee Toffee",\n    "description": "Coffee ice cream with fudge-covered toffee pieces provides a sweet and crunchy experience for coffee lovers.",\n    "original_rank": "25",\n    "region_num": "1"

In [14]:
#@title Turn Bracket Generation Response into DF and Run Checks
bracket_generation_df = (pd.DataFrame(
  json.loads(bracket_generation_response_json)).
  astype({
    'original_rank':'int',
    'region_num':'int',
    'seed':'int'
    })
  )

# Check # of items generated matches specification
assert len(bracket_generation_df['item']) == NUM_ITEMS

# Checks that # of unique items generated matches specification
assert bracket_generation_df['item'].nunique() == NUM_ITEMS

# Checks that # of regions generated matches specification
assert (len(bracket_generation_df[['region_num', 'region_name']].
  drop_duplicates()) == NUM_REGIONS)

# Check that # of items in each region matches specification
assert ((bracket_generation_df
  .groupby('region_num')
  .size()
  .reset_index(name='counts')['counts']
  == NUM_SEEDS_PER_REGION
  ).all())

# Check that there is 1 item for each region/seed combination
assert (bracket_generation_df
  .groupby(['region_num', 'region_name', 'seed'])
  .size()
  .reset_index(name='num_items')['num_items']
  == 1
  ).all()

display(bracket_generation_df)

AssertionError: 

In [15]:
#@title Alternative Way to Generate Bracket Directly from Ranked List
bracket_generation_df = ranked_list_df.copy()

# Divide items into seed by original rank into # of regions
bracket_generation_df['seed'] = ((bracket_generation_df['rank'] - 1) //
  NUM_REGIONS) + 1

# Divide items into region by "S-Curve" logic across seeds / original ranks
bracket_generation_df['region_num'] = np.where(
  # For odd seeds, follow overall rank mod # regions in order
  bracket_generation_df['seed'] % 2 == 1,
  ((bracket_generation_df['rank'] - 1) % NUM_REGIONS) + 1,
  # For even seeds, follow overall rank mod # regions in reverse order
  ((NUM_REGIONS - bracket_generation_df['rank']) % NUM_REGIONS) + 1
  )

bracket_generation_df['region_name'] = ("Region " +
  bracket_generation_df['region_num'].astype(str))

bracket_generation_df.rename(columns = {"rank": "original_rank"},
  inplace = True)

display(bracket_generation_df)

,item,description,original_rank,short_name,seed,region_num,region_name
0,Cherry Garcia,"A classic for a reason, this flavor combines cherry ice cream with cherries and fudge flakes, offering a simple yet satisfying flavor profile that has stood the test of time.",1,Cherry Gar,1,1,Region 1
1,Chocolate Chip Cookie Dough,"This beloved flavor features vanilla ice cream with gobs of chocolate chip cookie dough, a combination that appeals to a wide range of ice cream lovers with its classic mix-ins.",2,Choc Chip,1,2,Region 2
2,Half Baked,"Combining chocolate and vanilla ice creams with gobs of chocolate chip cookie dough and fudge brownies, this flavor offers a textural and flavorful experience that is both indulgent and comforting.",3,Half Baked,1,3,Region 3
3,The Tonight Dough,"With caramel and chocolate ice creams, chocolate cookie swirls, and gobs of chocolate chip cookie dough and peanut butter cookie dough, this flavor is a rich and complex treat that keeps you comin...",4,Tonight Dough,1,4,Region 4
4,Phish Food,"This flavor blends chocolate ice cream with gooey marshmallow swirls, caramel swirls, and fudge fish, creating a unique and playful combination of textures and flavors that is both fun and delicious.",5,Phish Food,2,4,Region 4
...,...,...,...,...,...,...,...
59,Chocolate Fudge Brownie (Non-Dairy),Chocolate non-dairy frozen dessert with fudge brownies.,60,Brownie ND,15,4,Region 4
60,Colin Kaepernick's Change the Whirled,"Caramel non-dairy frozen dessert with fudge chips, graham cracker swirls, and chocolate cookie swirls.",61,Change World,16,4,Region 4
61,Karamel Sutra Core (Non-Dairy),Chocolate and caramel non-dairy frozen dessert with fudge chips and a soft caramel core.,62,Sutra ND,16,3,Region 3
62,Lights! Caramel! Action! (Non-Dairy),"Vanilla non-dairy frozen dessert with salted caramel swirls, graham cracker swirls, and gobs of chocolate chip cookie dough.",63,Lights! ND,16,2,Region 2


In [16]:
#@title Show Generated Bracket with Items
bracket_generation_with_info = pd.merge(
  bracket_generation_df,
  (get_slot_seed_ordering_df(NUM_SEEDS_PER_REGION).
    rename(columns={'slot': 'region_slot'})
    ),
  on = "seed",
  how = "left"
  )

bracket_generation_with_info['overall_slot'] = ((
  bracket_generation_with_info['region_num'] - 1) * NUM_SEEDS_PER_REGION +
  bracket_generation_with_info['region_slot'])

bracket_generation_with_info['round_desc'] = NUM_ROUNDS

bracket_generation_with_info['round_slot'] = (
    bracket_generation_with_info['overall_slot'])

bracket_generation_with_info['display_text'] = (bracket_generation_with_info.
  apply(lambda row: f"{row['seed']} {row['short_name']}", axis=1)
  )

bracket_generation_with_info['hover_text'] = (bracket_generation_with_info.
  apply(lambda row:
    f"{row['item']}<br>"
    f"<b>Seed:</b> {row['seed']}<br>"
    f"<b>Region:</b> {row['region_name']}<br>"
    f"<b>Description:</b> {row['description']}<br>"
    f"<b>Original Overall Rank:</b> {row['original_rank']}<br>"
    ,
    axis=1
    )
  )

bracket_with_items_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_generation_with_info,
  empty_bracket_viz,
  trim_bracket_viz_vertically = False
  )

bracket_with_items_viz = bracket_with_items_viz_and_dfs['bracket_viz']

display(bracket_with_items_viz)

# Bracket Advancement
---

In [17]:
#@title Function to Get Matchup Winner from Gemini
def get_matchup_winner_from_gemini(matchup_dict, bracket_theme,
    advancement_special_criteria, grounding_source, provided_material_file_uri):

    matchup_winner_system_instruction = types.Part.from_text(text="""
    You are an expert at determining winners in head-to-head matchups in
    brackets - assessing each matchup and deciding who should win based on
    significant criteria related to the bracket theme. If relevant, use the
    reference material provided along with any specified advancement criteria to
    help determine the winner and winning margin (from 1 of 3 categories) as
    well as provide explanations for each matchup.
    """)

    matchup_winner_response_schema = {
      "type": "OBJECT",
      "properties": {
        "winning_item": {
          "type": "STRING",
          # Force it to return 1 of 2 items in matchup
          "enum": [matchup_dict['item_1'], matchup_dict['item_2']],
          "nullable": "False"
          },
        "winning_margin": {
          "type": "STRING",
          "enum": ['Close', 'Comfortable', 'Blowout'],
          "nullable": "False"
          },
        "explanation": {
          "type": "STRING",
          "nullable": "False"
          }
      },
      "required": [
        "winning_item",
        "winning_margin",
        "explanation"
      ]
    }

    matchup_winner_config = types.GenerateContentConfig(
      temperature = 1.0,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [matchup_winner_system_instruction],
      response_modalities = ["TEXT"],
      response_mime_type = "application/json",
      response_schema = matchup_winner_response_schema
      )

    matchup_winner_prompt = types.Part.from_text(text=f"""
    Given the following matchup in a {bracket_theme} bracket, determine the
    winner. Favor these special criteria in deciding the winner of each matchup:
    {advancement_special_criteria}.
    After prioritizing the special criteria, you can generally adhere
    to the seeds (lower seed numbers like 1 and 2 are more likely to win than
    higher seed numbers like 15 or 16), but also allow for some randomness (i.e.
    upsets) in the results as well. Provide a projected winning margin (Close,
    Comfortable, or Blowout) and short explanation (1 sentence) for why the
    winner advanced over the loser.
    Below is information on the matchup:
    {matchup_dict}
    """)

    content_generation_parts = [matchup_winner_prompt]

    if (grounding_source == "Provided Material" and provided_material_file_uri
        and provided_material_file_uri != ""):
        bracket_advancement_ref_material = types.Part.from_uri(
          file_uri=provided_material_file_uri,
          mime_type=get_mime_type_from_gcs_uri(provided_material_file_uri)
          )

        content_generation_parts = (content_generation_parts +
          [bracket_advancement_ref_material])

    matchup_winner_contents = [
      types.Content(
        role="user",
        parts=content_generation_parts
        )
      ]

    matchup_winner_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=matchup_winner_contents,
      config=matchup_winner_config
      )

    matchup_winner_response_json = matchup_winner_response.text

    return(matchup_winner_response_json)

In [18]:
#@title Get Bracket Matchups and Advancement by Round
original_bracket_df = bracket_generation_with_info.copy()

# Initialize this round to be NUM_ROUNDS
this_round = NUM_ROUNDS

# Initialize list to collect round-by-round-results
round_by_round_results = []

while this_round > 0:
    # Initialize this round bracket df to be copy of original bracket df
    if(this_round == NUM_ROUNDS):
      this_round_bracket_df = original_bracket_df.copy()
    # Otherwise filter original bracket df to teams that advanced in prev round
    else:
      this_round_bracket_df = (original_bracket_df[
        original_bracket_df['item'].isin(prev_round_results['winning_item'])].
        reset_index(drop = True)
        )
      del prev_round_results

    # Set to next round
    this_round_bracket_df['next_round_desc'] = (this_round - 1)

    # Next round slot based on ovr slot & # rounds into bracket (grouping by
    # powers of 2) - helps with determining next round matchups
    this_round_bracket_df['next_round_slot'] = np.ceil(
      this_round_bracket_df['overall_slot'] / 2**(NUM_ROUNDS -
        this_round_bracket_df['next_round_desc'])
      ).astype(int)

    this_round_bracket_df_for_merge = this_round_bracket_df[['item',
      'description', 'original_rank', 'short_name', 'seed', 'round_slot',
      'next_round_desc', 'next_round_slot']]

    this_round_matchups = pd.merge(
      this_round_bracket_df_for_merge,
      this_round_bracket_df_for_merge,
      on = ['next_round_desc', 'next_round_slot'],
      how = "inner",
      suffixes=('_1', '_2')
      )

    this_round_matchups = (this_round_matchups[
      this_round_matchups["round_slot_1"] < this_round_matchups["round_slot_2"]
      ].
      drop(["round_slot_1", "round_slot_2"], axis = 1).
      rename(columns = {
        'next_round_desc': 'round_desc',
        'next_round_slot': 'round_slot'
        }).
      sort_values(["round_slot"]).
      reset_index(drop = True)
      )

    # Original (non-parallel) calls to Gemini to get winner of each matchup
    this_round_matchups['gemini_matchup_analyis'] = this_round_matchups.apply(
      lambda row: get_matchup_winner_from_gemini(row.to_dict(), BRACKET_THEME,
        BRACKET_ADVANCEMENT_SPECIAL_CRITERIA, GROUNDING_SOURCE,
        PROVIDED_MATERIAL_FILE_URI),
      axis = 1
      )

    # Potential way to parallelize Gemini matchup analysis using Swifter
    # TODO: figure out better way to do this, potentially w/ async generation
    # this_round_matchups['gemini_matchup_analyis'] = (
    #   this_round_matchups.swifter.apply(
    #     lambda row: get_matchup_winner_from_gemini(row.to_dict(), BRACKET_THEME,
    #       BRACKET_ADVANCEMENT_SPECIAL_CRITERIA, GROUNDING_SOURCE,
    #       PROVIDED_MATERIAL_FILE_URI),
    #     axis=1
    #     )
    #   )

    this_round_matchups['gemini_matchup_analyis_dict'] = (
        this_round_matchups['gemini_matchup_analyis'].apply(json.loads))

    for col in ['winning_item', 'winning_margin', 'explanation']:
      this_round_matchups[col] = (
          this_round_matchups['gemini_matchup_analyis_dict'].apply(
              lambda x: x[col])
          )

    this_round_matchups['losing_item'] = np.where(
      this_round_matchups['winning_item'] == this_round_matchups['item_1'],
      this_round_matchups['item_2'],
      this_round_matchups['item_1']
      )

    this_round_results = this_round_matchups[['round_desc', 'round_slot',
      'winning_item', 'losing_item', 'winning_margin', 'explanation']]

    round_by_round_results = round_by_round_results + [this_round_results]

    print(f"Done with Round {this_round}")

    # Updating round count before next iteration of loop
    this_round = this_round - 1

    # Reassigning df to help w/ "this round" becoming "prev_round" in next round
    prev_round_results = this_round_results.copy()

    del this_round_results

all_round_results = pd.concat(round_by_round_results, ignore_index=True)

display(all_round_results)

Done with Round 6
Done with Round 5
Done with Round 4
Done with Round 3
Done with Round 2
Done with Round 1


,round_desc,round_slot,winning_item,losing_item,winning_margin,explanation
0,5,1,"""Milk"" & Cookies (Non-Dairy)",Cherry Garcia,Close,"""Milk"" & Cookies (Non-Dairy) contains chocolate chip cookies which are an important component of cookie dough, so it wins this matchup."
1,5,2,Thick Mint,Strawberry Topped Tart,Close,Thick Mint has chocolate cookie swirls which edges out the lesser-desired pie crust pieces and sprinkles of Strawberry Topped Tart.
2,5,3,Mint Chocolate Cookie,Bossin' Cream Pie,Close,Mint Chocolate Cookie wins due to its peppermint ice cream and chocolate sandwich cookies combo.
3,5,4,Chunky Monkey,Pecan Pie,Close,"Despite its lower seed, Pecan Pie, a Limited Batch flavor, seems to have a good combination of flavors, however Chunky Monkey is still the favorite as a classic flavor."
4,5,5,"Coffee, Coffee BuzzBuzzBuzz!",Cannoli,Close,"Coffee Buzz has espresso bean fudge chunks, giving it a slight edge over Cannoli and allowing it to advance."
...,...,...,...,...,...,...
58,2,3,Chocolate Chip Cookie Dough (Non-Dairy),Half Baked,Close,"Even though it is non-dairy, Chocolate Chip Cookie Dough is the GOAT and will pull off the upset over Half Baked due to the special criteria."
59,2,4,The Tonight Dough,Milk & Cookies,Comfortable,"The Tonight Dough has cookie dough and peanut butter cookie dough, which is a huge advantage."
60,1,1,Chocolate Chip Cookie Dough,"""Milk"" & Cookies (Non-Dairy)",Comfortable,"Even though ""Milk"" & Cookies has cookie elements, Chocolate Chip Cookie Dough is the better version and the #1 seed, giving it the edge in this matchup."
61,1,2,The Tonight Dough,Chocolate Chip Cookie Dough (Non-Dairy),Comfortable,"The Tonight Dough is the better flavor since it is the #1 seed and it contains cookie dough (and peanut butter cookie dough, which is a bonus)."


In [19]:
#@title Bracket Advancement Directly from Gemini over Original Bracket

def get_bracket_advancement_from_original_bracket(bracket_theme, num_items,
    bracket_df, grounding_source, provided_material_file_uri):

    bracket_advancement_system_instruction = types.Part.from_text(text="""
    You are an expert at filling out brackets - assessing each matchup and
    deciding who should win based on significant criteria related to the bracket
    theme. If relevant, use the reference material provided to help determine
    the winner and provide explanations for each matchup.
    """)

    bracket_json_records = bracket_df.to_json(orient = 'records')

    bracket_matchup_rounds = (BRACKET_ROUNDS[
      (BRACKET_ROUNDS['num_items_in_round'] <= num_items) &
      (BRACKET_ROUNDS['num_items_in_round'] > 1)
      ]['round_name'].to_list()
      )

    bracket_advancement_response_schema = {
      "type": "ARRAY",
      "items": {
        "type": "OBJECT",
        "properties": {
          "round": {
            "type": "STRING",
            "enum": bracket_matchup_rounds,
            "nullable": "False"
          },
          "winning_item": {
            "type": "STRING",
            "nullable": "False"
          },
          "losing_item": {
            "type": "STRING",
            "nullable": "False"
          },
          "explanation": {
            "type": "STRING",
            "nullable": "False"
          }
        },
        "required": [
          "round",
          "winning_item",
          "losing_item",
          "explanation"
        ]
      }
    }

    bracket_advancement_config = types.GenerateContentConfig(
      temperature = 1.0,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [bracket_advancement_system_instruction],
      response_modalities = ["TEXT"],
      response_mime_type = "application/json",
      response_schema = bracket_advancement_response_schema
      )

    bracket_advancement_prompt = types.Part.from_text(text=f"""
    Given the following bracket, play out each matchup by round, advancing
    winners based on matchups round by round through the final, and then
    crowning a single champion. Favor these special criteria in deciding the
    winner of each matchup:
    {BRACKET_ADVANCEMENT_SPECIAL_CRITERIA}.
    Provide an explanation for why the winner advanced over the loser in each
    matchup. After prioritizing the special criteria, you can generally adhere
    to the seeds (lower seed numbers like 1 and 2 are more likely to win than
    higher seed numbers like 15 or 16), but also allow for some randomness (like
    upsets) in the results as well. When there is a big upset, call it out in
    the explanation.
    Make sure to return a winner and loser for each of the {num_items - 1}
    matchups in the bracket. There should be exactly {num_items - 1} results
    in the response.
    Below is the bracket:
    {bracket_json_records}
    """)

    content_generation_parts = [bracket_advancement_prompt]

    if (grounding_source == "Provided Material" and provided_material_file_uri
        and provided_material_file_uri != ""):
        bracket_advancement_ref_material = types.Part.from_uri(
          file_uri=provided_material_file_uri,
          mime_type=get_mime_type_from_gcs_uri(provided_material_file_uri)
          )

        content_generation_parts = (content_generation_parts +
          [bracket_advancement_ref_material])

    bracket_advancement_contents = [
      types.Content(
        role="user",
        parts=content_generation_parts
        )
      ]

    bracket_advancement_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=bracket_advancement_contents,
      config=bracket_advancement_config
      )

    bracket_advancement_response_json = bracket_advancement_response.text

    return(bracket_advancement_response_json)

bracket_advancement_response_json = (
  get_bracket_advancement_from_original_bracket(
    bracket_theme = BRACKET_THEME,
    num_items = NUM_ITEMS,
    bracket_df = bracket_generation_df,
    grounding_source = GROUNDING_SOURCE,
    provided_material_file_uri = PROVIDED_MATERIAL_FILE_URI
    )
  )

display(bracket_advancement_response_json)

'[\n  {\n    "round": "Round of 64",\n    "winning_item": "Cherry Garcia",\n    "losing_item": "Milk & Cookies (Non-Dairy)",\n    "explanation": "Cherry Garcia is a classic and wins against a non-dairy option."\n  },\n  {\n    "round": "Round of 64",\n    "winning_item": "Thick Mint",\n    "losing_item": "Banana Foster Core (Non-Dairy)",\n    "explanation": "Thick Mint is more appealing than a non-dairy banana cinnamon flavor."\n  },\n  {\n    "round": "Round of 64",\n    "winning_item": "Brownie Batter Core",\n    "losing_item": "Coffee Toffee Bar Crunch",\n    "explanation": "Brownie Batter Core offers a more direct brownie experience which is slightly more appealing."\n  },\n  {\n    "round": "Round of 64",\n    "winning_item": "Coffee, Coffee BuzzBuzzBuzz!",\n    "losing_item": "Pecan Pie",\n    "explanation": "Coffee Buzz is a solid flavor and edges out the limited batch Pecan Pie."\n  },\n  {\n    "round": "Round of 64",\n    "winning_item": "New York Super Fudge Chunk",\n    "lo

In [20]:
#@title Turn Bracket Advancement Response into DF and Run Checks
bracket_advancement_df = (pd.DataFrame(
  json.loads(bracket_advancement_response_json))
  [['round', 'winning_item', 'losing_item', 'explanation']]
  )

display(bracket_advancement_df)

,round,winning_item,losing_item,explanation
0,Round of 64,Cherry Garcia,Milk & Cookies (Non-Dairy),Cherry Garcia is a classic and wins against a non-dairy option.
1,Round of 64,Thick Mint,Banana Foster Core (Non-Dairy),Thick Mint is more appealing than a non-dairy banana cinnamon flavor.
2,Round of 64,Brownie Batter Core,Coffee Toffee Bar Crunch,Brownie Batter Core offers a more direct brownie experience which is slightly more appealing.
3,Round of 64,"Coffee, Coffee BuzzBuzzBuzz!",Pecan Pie,Coffee Buzz is a solid flavor and edges out the limited batch Pecan Pie.
4,Round of 64,New York Super Fudge Chunk,Boom Chocolatta! Core (Non-Dairy),NY Super Fudge Chunk is the superior chocolate and nut combination compared to the non-dairy Boom Chocolatta.
...,...,...,...,...
60,Round of 16,The Tonight Dough,Mint Chocolate Chance,The Tonight Dough has the perfect mix of everything.
61,Quarterfinals,The Tonight Dough,Chocolate Fudge Brownie,The Tonight Dough has cookie dough and peanut butter.
62,Quarterfinals,Half Baked,Peanut Butter World,"Half Baked has cookie dough and brownies, wins over peanut butter."
63,Semifinals,Half Baked,The Tonight Dough,Half Baked has peanut butter and cookie dough and brownies.


In [21]:
bracket_advancement_df['round'].value_counts()

,count
round,
Round of 64,40
Round of 32,12
Round of 16,7
Quarterfinals,3
Semifinals,2
Final,1


In [22]:
#@title Prepare Full Bracket Advancement from Gemini for Showing on Bracket
bracket_advancement_with_info = pd.merge(
  pd.merge(
    bracket_advancement_df[['round', 'winning_item', 'losing_item',
        'explanation']].
      rename(columns = {"winning_item": "item", "round": "matchup_round"}),
    BRACKET_ROUNDS[['round_desc', 'round_name']].assign(
      round_desc = lambda x: (x['round_desc'] - 1)
      ).
      rename(columns = {"round_name": "matchup_round"}),
    on = 'matchup_round',
    how = 'left'
    ).
    rename(columns = {"matchup_round": "prev_matchup_round",
      "losing_item": "defeated_in_prev_round"}
      ),
  bracket_generation_with_info[['item', 'short_name', 'original_rank', 'seed',
    'overall_slot', 'display_text']],
  on = 'item',
  how = 'left'
  )

# Round slot based on ovr slot & # rounds into bracket (grouping by powers of 2)
bracket_advancement_with_info['round_slot'] = np.ceil(
  bracket_advancement_with_info['overall_slot'] / 2**(NUM_ROUNDS -
    bracket_advancement_with_info['round_desc'] )
  ).astype(int)

bracket_advancement_with_info['hover_text'] = (bracket_advancement_with_info.
  apply(lambda row:
    f"{row['item']}<br>"
    f"<b>{row['prev_matchup_round']}: </b>"
    f"defeated {row['defeated_in_prev_round']}<br>"
    f"{row['explanation']}<br>"
    f"<b>Original Overall Rank:</b> {row['original_rank']}<br>"
    ,
    axis=1
    )
  )

In [23]:
#@title Prepare Round-by-Round Bracket Advancement for Showing on Bracket
bracket_advancement_with_info = pd.merge(
  pd.merge(
    # Merge results with bracket rounds to get more info on previous round
    pd.merge(
      all_round_results[['round_desc', 'round_slot', 'winning_item',
          'losing_item', 'winning_margin', 'explanation']].
        assign(
          prev_round_desc = lambda x: (x['round_desc'] + 1)
          ).
        rename(columns = {"winning_item": "item"}),
      BRACKET_ROUNDS[['round_desc', 'round_name']].
        rename(columns = {
          "round_desc": "prev_round_desc",
          "round_name": "prev_matchup_round"
          }),
      on = 'prev_round_desc',
      how = 'left'
      ),
    # Merge with bracket info on winning item to use in bracket
    bracket_generation_with_info[['item', 'short_name', 'original_rank', 'seed',
      'overall_slot', 'display_text']],
    on = 'item',
    how = 'left'
    ),
    # Merge with bracket info on losing item to use in bracket text
    bracket_generation_with_info[['item', 'seed']].
      rename(columns = {"item": "losing_item", "seed": "losing_seed"}),
    on = 'losing_item',
    how = 'left'
  )

bracket_advancement_with_info['hover_text'] = (bracket_advancement_with_info.
  apply(lambda row:
    f"{row['seed']} {row['item']}<br>"
    f"{row['prev_matchup_round']}: defeated {row['losing_seed']} "
    f"{row['losing_item']} ({row['winning_margin']})<br>"
    f"{row['explanation']}"
    ,
    axis=1
    )
  )

display(bracket_advancement_with_info)

,round_desc,round_slot,item,losing_item,winning_margin,explanation,prev_round_desc,prev_matchup_round,short_name,original_rank,seed,overall_slot,display_text,losing_seed,hover_text
0,5,1,"""Milk"" & Cookies (Non-Dairy)",Cherry Garcia,Close,"""Milk"" & Cookies (Non-Dairy) contains chocolate chip cookies which are an important component of cookie dough, so it wins this matchup.",6,Round of 64,Milk ND,64,16,2,16 Milk ND,1,"16 ""Milk"" & Cookies (Non-Dairy)<br>Round of 64: defeated 1 Cherry Garcia (Close)<br>""Milk"" & Cookies (Non-Dairy) contains chocolate chip cookies which are an important component of cookie dough, s..."
1,5,2,Thick Mint,Strawberry Topped Tart,Close,Thick Mint has chocolate cookie swirls which edges out the lesser-desired pie crust pieces and sprinkles of Strawberry Topped Tart.,6,Round of 64,Thick Mint,33,9,4,9 Thick Mint,8,9 Thick Mint<br>Round of 64: defeated 8 Strawberry Topped Tart (Close)<br>Thick Mint has chocolate cookie swirls which edges out the lesser-desired pie crust pieces and sprinkles of Strawberry Top...
2,5,3,Mint Chocolate Cookie,Bossin' Cream Pie,Close,Mint Chocolate Cookie wins due to its peppermint ice cream and chocolate sandwich cookies combo.,6,Round of 64,Mint Choc,17,5,5,5 Mint Choc,12,5 Mint Chocolate Cookie<br>Round of 64: defeated 12 Bossin' Cream Pie (Close)<br>Mint Chocolate Cookie wins due to its peppermint ice cream and chocolate sandwich cookies combo.
3,5,4,Chunky Monkey,Pecan Pie,Close,"Despite its lower seed, Pecan Pie, a Limited Batch flavor, seems to have a good combination of flavors, however Chunky Monkey is still the favorite as a classic flavor.",6,Round of 64,Chunky Monk,16,4,7,4 Chunky Monk,13,"4 Chunky Monkey<br>Round of 64: defeated 13 Pecan Pie (Close)<br>Despite its lower seed, Pecan Pie, a Limited Batch flavor, seems to have a good combination of flavors, however Chunky Monkey is st..."
4,5,5,"Coffee, Coffee BuzzBuzzBuzz!",Cannoli,Close,"Coffee Buzz has espresso bean fudge chunks, giving it a slight edge over Cannoli and allowing it to advance.",6,Round of 64,Coffee Buzz,24,6,9,6 Coffee Buzz,11,"6 Coffee, Coffee BuzzBuzzBuzz!<br>Round of 64: defeated 11 Cannoli (Close)<br>Coffee Buzz has espresso bean fudge chunks, giving it a slight edge over Cannoli and allowing it to advance."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2,3,Chocolate Chip Cookie Dough (Non-Dairy),Half Baked,Close,"Even though it is non-dairy, Chocolate Chip Cookie Dough is the GOAT and will pull off the upset over Half Baked due to the special criteria.",3,Quarterfinals,Chip ND,59,15,48,15 Chip ND,1,"15 Chocolate Chip Cookie Dough (Non-Dairy)<br>Quarterfinals: defeated 1 Half Baked (Close)<br>Even though it is non-dairy, Chocolate Chip Cookie Dough is the GOAT and will pull off the upset over ..."
59,2,4,The Tonight Dough,Milk & Cookies,Comfortable,"The Tonight Dough has cookie dough and peanut butter cookie dough, which is a huge advantage.",3,Quarterfinals,Tonight Dough,4,1,49,1 Tonight Dough,6,"1 The Tonight Dough<br>Quarterfinals: defeated 6 Milk & Cookies (Comfortable)<br>The Tonight Dough has cookie dough and peanut butter cookie dough, which is a huge advantage."
60,1,1,Chocolate Chip Cookie Dough,"""Milk"" & Cookies (Non-Dairy)",Comfortable,"Even though ""Milk"" & Cookies has cookie elements, Chocolate Chip Cookie Dough is the better version and the #1 seed, giving it the edge in this matchup.",2,Semifinals,Choc Chip,2,1,17,1 Choc Chip,16,"1 Chocolate Chip Cookie Dough<br>Semifinals: defeated 16 ""Milk"" & Cookies (Non-Dairy) (Comfortable)<br>Even though ""Milk"" & Cookies has cookie elements, Chocolate Chip Cookie Dough is the better v..."
61,1,2,The Tonight Dough,Chocolate Chip Cookie Dough (Non-Dairy),Comfortable,"The Tonight Dough is the better flavor since it is the #1 seed and it contains cookie dough (and peanut butter cookie dough, which is a bonus).",2,Semifinals,Tonight Dough,4,1,49,1 Tonight Dough,15,1 The Tonight Dough<br>Semifinals: defeated 15 Ch

In [24]:
#@title Show Generated Bracket with Advancement
bracket_with_winners_viz_and_dfs = build_bracket_viz(
  NUM_ITEMS,
  BRACKET_ROUNDS,
  BRACKET_TITLE,
  BRACKET_FOOTER,
  bracket_advancement_with_info,
  bracket_with_items_viz,
  trim_bracket_viz_vertically = False
  )

bracket_with_winners_viz = bracket_with_winners_viz_and_dfs['bracket_viz']

display(bracket_with_winners_viz)

# Add Celebratory Image for Champion to Bracket
---

In [25]:
#@title Functions to Create Celebratory Image for Champion and Add to Bracket
bracket_champion = (bracket_advancement_with_info[
  bracket_advancement_with_info['round_desc'] == 0]['item'].iloc[0])

def get_bracket_champion_image(bracket_theme, num_items, bracket_champion):
    try:
        bracket_champion_imagen_prompt = f"""
        Celebratory photo of {bracket_champion} as the champion,
        emerging from a {num_items}-item bracket of {bracket_theme}
        """
        bracket_champion_image_gen = GENAI_CLIENT.models.generate_images(
          model=IMAGEN_MODEL_ID,
          prompt=bracket_champion_imagen_prompt,
          config=types.GenerateImagesConfig(
            number_of_images=1,
            aspect_ratio="1:1",
            person_generation="ALLOW_ADULT",
            safety_filter_level="BLOCK_LOW_AND_ABOVE",
            enhance_prompt=True,
            include_rai_reason=True,
            include_safety_attributes=True
            )
        )

        bracket_champion_image = (bracket_champion_image_gen.
          generated_images[0].image)

        display_image(bracket_champion_image)

    except:
        bracket_champion_imagen_prompt = f"""
        Celebratory photo of a single item from the theme of {bracket_theme}
        as the champion, emerging from a {num_items}-item bracket
        """
        print(
          "Image generation failed with original prompt, trying more generic:"
          f"{bracket_champion_imagen_prompt}"
          )

        bracket_champion_image_gen = GENAI_CLIENT.models.generate_images(
          model=IMAGEN_MODEL_ID,
          prompt=bracket_champion_imagen_prompt,
          config=types.GenerateImagesConfig(
            number_of_images=1,
            aspect_ratio="1:1",
            person_generation="ALLOW_ADULT",
            safety_filter_level="BLOCK_LOW_AND_ABOVE",
            enhance_prompt=True,
            include_rai_reason=True,
            include_safety_attributes=True
            )
          )

        bracket_champion_image = (bracket_champion_image_gen.
          generated_images[0].image)

        display_image(bracket_champion_image)

    return bracket_champion_image

def add_bracket_champion_image_to_bracket(exisiting_bracket_viz, bracket_df,
    bracket_champion_image, num_items):

    # Get some image parameters related to num items
    BrCo = get_bracket_viz_config(num_items)

    bracket_champion_image._pil_image.save("bracket_champion.png")

    bracket_champion_image_base64 = get_image_base64("bracket_champion.png")

    bracket_df_champ_info = bracket_df[bracket_df['round_desc'] == 0].iloc[0]

    bracket_viz_with_champ_image = copy.deepcopy(exisiting_bracket_viz)

    bracket_viz_with_champ_image.add_layout_image(
      dict(
        source='data:image/png;base64,' + bracket_champion_image_base64,
        xref="x",
        yref="y",
        x=bracket_df_champ_info['rect_mid_x'],
        y=bracket_df_champ_info['rect_bottom_y'] - BrCo["image_space_vs_min_y"],
        sizex=BrCo["image_width"],
        sizey=BrCo["image_height"],
        xanchor="center",
        yanchor="top"
        )
      )

    return(bracket_viz_with_champ_image)

In [26]:
#@title Add Champion Celebratory Image to Bracket
# Wrap image generation and addition to bracket in case it fails
try:
    bracket_champion_image = get_bracket_champion_image(
      bracket_theme = BRACKET_THEME,
      num_items = NUM_ITEMS,
      bracket_champion = bracket_champion
      )

    clear_output()

    bracket_with_champ_image_viz = add_bracket_champion_image_to_bracket(
      bracket_with_winners_viz,
      bracket_with_winners_viz_and_dfs['bracket_dfs']['bracket_df'],
      bracket_champion_image,
      NUM_ITEMS
      )

except:
    print("Image generation and addition to bracket failed, proceed without it")
    bracket_with_champ_image_viz = bracket_with_winners_viz

display(bracket_with_champ_image_viz)

# Get Bracket Round-by-Round Results as Text and Gemini Summary
---

In [27]:
#@title Functions to Get Text Pieces from Bracket Results
def get_bracket_items_text_from_df(bracket_df):
    bracket_items_text = "##Bracket Items and Descriptions##\n\n"

    for region in sorted(bracket_df['region_name'].unique()):
      bracket_items_text += f"###{region}###\n\n"
      for index, row in (
        bracket_df[bracket_df['region_name'] == region].
        sort_values("seed").
        iterrows()
        ):
        bracket_items_text += (
          f"**{row['seed']}) {row['item']}** (Orig Rk #"
          f"{row['original_rank']}): {row['description']}\n\n"
          )
      bracket_items_text += "\n"

    return(bracket_items_text)

def get_bracket_results_text_from_df(bracket_results_df):
    bracket_df_grouped_results = bracket_results_df.groupby(
      ['prev_round_desc', 'prev_matchup_round'])

    # Store group keys in a list
    bracket_df_group_keys = list(bracket_df_grouped_results.groups.keys())

    # Combine results into a single string
    round_by_round_results_text = '##Round-by-Round Results##\n\n'

    # Iterate through group keys in reverse order
    for round_name in reversed(bracket_df_group_keys):
        round_data = bracket_df_grouped_results.get_group(round_name)
        round_by_round_results_text += f"\n\n###{round_name[1]}###\n"
        for index, row in round_data.iterrows():
            round_by_round_results_text += (
              f"**{row['seed']} {row['item']} def. {row['losing_seed']} "
              f"{row['losing_item']}** ({row['winning_margin']}): "
              f"{row['explanation']}\n\n"
              )

    return(round_by_round_results_text)

def get_bracket_gemini_summary(bracket_title, bracket_info_and_results_text):

    bracket_summary_system_instruction = types.Part.from_text(text="""
    You are an expert at summarizing the results from a competition among
    various items that can be organized into a bracket form. In your summaries,
    focus on the items that advanced further more and the overall champion that
    emerged at the end of the bracket the most.
    """)

    bracket_summary_config = types.GenerateContentConfig(
      temperature = 1.0,
      top_p = 0.95,
      max_output_tokens = 8192,
      system_instruction = [bracket_summary_system_instruction],
      response_modalities = ["TEXT"]
      )

    bracket_summary_prompt = types.Part.from_text(text=f"""
    Summarize the {bracket_title} using the information and results below in
    about 250 words.
    Bracket info and results:
    {bracket_info_and_results_text}
    """)

    bracket_summary_response = GENAI_CLIENT.models.generate_content(
      model=GEMINI_MODEL_ID,
      contents=bracket_summary_prompt,
      config=bracket_summary_config
      )

    bracket_summary_text = ("##Bracket Results Summary from Gemini##\n\n"
      f"{bracket_summary_response.text}")

    return(bracket_summary_text)

def get_all_bracket_text(bracket_title, bracket_df, bracket_results_df):

    bracket_items_text = get_bracket_items_text_from_df(bracket_df)

    round_results_text = get_bracket_results_text_from_df(bracket_results_df)

    bracket_items_results_text = (f"{bracket_items_text}\n\n\n"
      f"{round_results_text}")

    bracket_gemini_summary = get_bracket_gemini_summary(
        bracket_title, bracket_items_results_text)

    all_combined_text = (
      f"#{bracket_title}#\n\n"
      f"{bracket_gemini_summary}\n\n\n"
      f"{bracket_items_results_text}"
      )

    return(all_combined_text)

all_bracket_text = get_all_bracket_text(
  bracket_title = BRACKET_TITLE,
  bracket_df = bracket_generation_with_info,
  bracket_results_df = bracket_advancement_with_info
  )

display(Markdown(all_bracket_text))

#Ben & Jerry's Ice Cream Flavors Bracket from Gemini#

##Bracket Results Summary from Gemini##

The Ben & Jerry's Ice Cream Flavors Bracket saw some intense competition, with classic and innovative flavors battling it out. In Region 1, "Milk" & Cookies (Non-Dairy) had a remarkable run as a 16-seed, all the way to the Quarterfinals, defeating the top-seeded Cherry Garcia in the first round, showcasing the power of cookie-based flavors. Brownie Batter Core also emerged as a strong contender, reaching the Round of 16. In Region 2, Chocolate Chip Cookie Dough lived up to its top-seed status, consistently advancing with its classic appeal. However, Marshmallow Sky was the upstart in this group, getting to the Round of 32. Region 3 witnessed Chocolate Chip Cookie Dough (Non-Dairy) defying its 15-seed rank and reaching the Semifinals, proving that non-dairy options can stand strong. Lastly, The Tonight Dough dominated Region 4, with its combination of caramel and chocolate ice creams and cookie dough mix-ins.

Ultimately, the final showdown pitted two top contenders against each other: the classic Chocolate Chip Cookie Dough and the innovative The Tonight Dough. In a closely contested battle, Chocolate Chip Cookie Dough emerged as the champion, proving that the original is often the best, thus validating it as the supreme Ben & Jerry's ice cream flavor.



##Bracket Items and Descriptions##

###Region 1###

**1) Cherry Garcia** (Orig Rk #1): A classic for a reason, this flavor combines cherry ice cream with cherries and fudge flakes, offering a simple yet satisfying flavor profile that has stood the test of time.

**2) New York Super Fudge Chunk** (Orig Rk #8): This flavor packs a punch with chocolate ice cream, white and dark fudge chunks, pecans, walnuts, and fudge-covered almonds, delivering a nutty and chocolatey experience.

**3) Karamel Sutra Core** (Orig Rk #9): Chocolate and caramel ice creams with fudge chips and a soft caramel core provide a balanced mix of sweet and rich flavors with a satisfying gooey center.

**4) Chunky Monkey** (Orig Rk #16): Banana ice cream with fudge chunks and walnuts appeals to those who enjoy a fruity twist with nutty and chocolatey additions.

**5) Mint Chocolate Cookie** (Orig Rk #17): Peppermint ice cream with chocolate sandwich cookies is a refreshing option for mint lovers.

**6) Coffee, Coffee BuzzBuzzBuzz!** (Orig Rk #24): Coffee ice cream with espresso bean fudge chunks is a caffeinated treat for coffee enthusiasts.

**7) Coffee Toffee Bar Crunch** (Orig Rk #25): Coffee ice cream with fudge-covered toffee pieces provides a sweet and crunchy experience for coffee lovers.

**8) Strawberry Topped Tart** (Orig Rk #32): Sweet cream ice cream with strawberry swirls and pie crust pieces topped with white chocolatey ganache and candy sprinkles mixes strawberry and candy flavors.

**9) Thick Mint** (Orig Rk #33): Mint ice cream with chocolate cookie swirls and mint chocolate cookie balls topped with chocolatey ganache and chocolate cookies adds some extra mint and chocolate.

**10) Brownie Batter Core** (Orig Rk #40): Chocolate and vanilla ice creams with fudge brownies and a brownie batter core.

**11) Cannoli** (Orig Rk #41): Mascarpone ice cream with fudge-covered pastry shell pieces and mascarpone swirls (New).

**12) Bossin' Cream Pie** (Orig Rk #48): Vanilla custard ice cream with cake pieces and pastry cream swirls topped with milk chocolatey ganache and fudge chips (Topped).

**13) Pecan Pie** (Orig Rk #49): Buttery ice cream with pecans, pie crust pieces, and thick caramel swirls (Limited Batch, New).

**14) Bananas Foster Core (Non-Dairy)** (Orig Rk #56): Banana and cinnamon non-dairy frozen dessert with almond toffee pieces and a salted caramel core.

**15) Boom Chocolatta! Core (Non-Dairy)** (Orig Rk #57): Mocha and caramel non-dairy frozen dessert with fudge flakes, gluten-free chocolate cookies, and a gluten-free chocolate cookie core.

**16) "Milk" & Cookies (Non-Dairy)** (Orig Rk #64): Vanilla non-dairy frozen dessert with chocolate chip cookies, chocolate sandwich cookies, and chocolate cookie swirls.


###Region 2###

**1) Chocolate Chip Cookie Dough** (Orig Rk #2): This beloved flavor features vanilla ice cream with gobs of chocolate chip cookie dough, a combination that appeals to a wide range of ice cream lovers with its classic mix-ins.

**2) Americone Dream** (Orig Rk #7): Vanilla ice cream with fudge-covered waffle cone pieces and a caramel swirl offers a delightful combination of textures and flavors reminiscent of a classic ice cream cone.

**3) Everything But The...** (Orig Rk #10): Chocolate and vanilla ice creams with peanut butter cups, fudge-covered toffee pieces, white chocolatey chunks, and fudge-covered almonds offer a complex and varied assortment of mix-ins for a truly indulgent experience.

**4) Boom Chocolatta! Core** (Orig Rk #15): Mocha and caramel ice creams with chocolate cookies, fudge flakes, and a chocolate cookie core provide a layered and chocolatey experience.

**5) Minter Wonderland** (Orig Rk #18): Dark chocolate mint ice cream with marshmallow swirls and chocolate cookie swirls combines rich chocolate with minty freshness.

**6) Chocolate Peanut Butter Split** (Orig Rk #23): Chocolate and banana ice creams with mini peanut butter cups provide a fruity and nutty flavor combination.

**7) Salted Caramel Brownie** (Orig Rk #26): Vanilla ice cream with salted caramel swirls and fudge brownies topped with chocolatey ganache and caramel cups blends multiple flavors into one cup.

**8) Dirt Cake** (Orig Rk #31): Vanilla pudding ice cream with chocolate sandwich cookies and chocolate cookie swirls topped with milk chocolatey ganache and chocolate cookie crumble resembles the dirt cake dessert.

**9) Whiskey Biz** (Orig Rk #34): Brown butter bourbon ice cream with blonde brownies and whiskey caramel swirls topped with white chocolatey ganache and white fudge chunks includes a hint of bourbon.

**10) Pumpkin Cheesecake** (Orig Rk #39): Pumpkin cheesecake ice cream with a thick graham cracker swirl (Seasonal).

**11) Chewy Gooey Cookie** (Orig Rk #42): Milk chocolate and coconut ice creams with fudge flakes, shortbread cookies, and caramel swirls.

**12) Tiramisu** (Orig Rk #47): Mascarpone ice cream with fudge swirls and shortbread pieces topped with chocolatey ganache and espresso fudge chunks (New).

**13) Marshmallow Sky** (Orig Rk #50): Marshmallow ice cream with marshmallow swirls and gobs of chocolate chip cookie dough and chocolate chocolate chip cookie dough (Limited Batch, New).

**14) Americone Dream (Non-Dairy)** (Orig Rk #55): Vanilla non-dairy frozen dessert with fudge-covered waffle cone pieces and a caramel swirl.

**15) Cherry Garcia (Non-Dairy)** (Orig Rk #58): Cherry non-dairy frozen dessert with cherries and fudge flakes.

**16) Lights! Caramel! Action! (Non-Dairy)** (Orig Rk #63): Vanilla non-dairy frozen dessert with salted caramel swirls, graham cracker swirls, and gobs of chocolate chip cookie dough.


###Region 3###

**1) Half Baked** (Orig Rk #3): Combining chocolate and vanilla ice creams with gobs of chocolate chip cookie dough and fudge brownies, this flavor offers a textural and flavorful experience that is both indulgent and comforting.

**2) Chocolate Fudge Brownie** (Orig Rk #6): Chocolate ice cream with fudge brownies is a straightforward and intensely chocolatey option for those who prefer a simpler, yet deeply satisfying flavor.

**3) Netflix & Chilll'd** (Orig Rk #11): Peanut butter ice cream with sweet and salty pretzel swirls and fudge brownies caters to those who enjoy a balance of sweet and savory elements.

**4) Salted Caramel Core** (Orig Rk #14): Sweet cream ice cream with blonde brownies and a salted caramel core combines classic flavors with a gooey, salted caramel center.

**5) Oat of This Swirled** (Orig Rk #19): Buttery brown sugar ice cream with fudge flakes and oatmeal cinnamon cookie swirls delivers a cozy and comforting flavor.

**6) Strawberry Cheesecake** (Orig Rk #22): Strawberry cheesecake ice cream with strawberries and a thick graham cracker swirl captures the essence of the beloved dessert.

**7) PB Over the Top** (Orig Rk #27): Chocolate ice cream with peanut butter swirls and peanut butter cups topped with chocolatey ganache and mini peanut butter cups gives an extra peanut buttery taste.

**8) Chocolate Milk & Cookies** (Orig Rk #30): Chocolate ice cream with chocolate chip cookies and chocolate cookie swirls topped with milk chocolatey ganache and fudge chips creates a creamy chocolate cup.

**9) Churray For Churros!** (Orig Rk #35): Buttery cinnamon ice cream with churro pieces and crunchy cinnamon swirls.

**10) Pistachio Pistachio** (Orig Rk #38): Pistachio ice cream with lightly roasted pistachios.

**11) Mint Chocolate Chance** (Orig Rk #43): Mint ice cream loaded with fudge brownies.

**12) Peanut Butter World** (Orig Rk #46): Milk chocolate ice cream with peanut buttery swirls and chocolate cookie swirls (Target).

**13) To Be Announced** (Orig Rk #51): Another great flavor, coming soon! (Limited Batch, New)

**14) Impretzively Fudged** (Orig Rk #54): Chocolate ice cream with fudge-covered pretzel pieces and pretzel swirls (New).

**15) Chocolate Chip Cookie Dough (Non-Dairy)** (Orig Rk #59): Vanilla non-dairy frozen dessert with gobs of chocolate chip cookie dough and fudge flakes.

**16) Karamel Sutra Core (Non-Dairy)** (Orig Rk #62): Chocolate and caramel non-dairy frozen dessert with fudge chips and a soft caramel core.


###Region 4###

**1) The Tonight Dough** (Orig Rk #4): With caramel and chocolate ice creams, chocolate cookie swirls, and gobs of chocolate chip cookie dough and peanut butter cookie dough, this flavor is a rich and complex treat that keeps you coming back for more.

**2) Phish Food** (Orig Rk #5): This flavor blends chocolate ice cream with gooey marshmallow swirls, caramel swirls, and fudge fish, creating a unique and playful combination of textures and flavors that is both fun and delicious.

**3) Peanut Butter Cup** (Orig Rk #12): Peanut butter ice cream with peanut butter cups is a straightforward option for peanut butter lovers, delivering a rich and creamy experience.

**4) Lights! Caramel! Action!** (Orig Rk #13): Vanilla ice cream with salted caramel swirls, graham cracker swirls, and gobs of chocolate chip cookie dough offers a blend of sweet, salty, and crunchy elements.

**5) Chocolate Therapy** (Orig Rk #20): Chocolate ice cream with chocolate cookies and swirls of chocolate pudding ice cream is an intensely chocolatey experience.

**6) Milk & Cookies** (Orig Rk #21): Vanilla ice cream with a chocolate cookie swirl, chocolate chip, and chocolate chocolate chip cookies offers a classic and comforting combination.

**7) Raspberry Cheesecake** (Orig Rk #28): Cheesecake ice cream with raspberry swirls and graham cracker pieces topped with white chocolatey ganache and graham cracker crumble mixes the flavors of cheesecake and raspberry.

**8) Chocolate Caramel Cookie Dough** (Orig Rk #29): Chocolate ice cream with caramel swirls and gobs of chocolate chip cookie dough topped with chocolatey ganache and caramel cups adds some extra chocolate.

**9) Dublin Mudslide** (Orig Rk #36): Irish cream ice cream with chocolate chocolate chip cookies and coffee fudge swirls.

**10) Gimme S'more!** (Orig Rk #37): Toasted marshmallow ice cream with chocolate cookie swirls, graham cracker swirls, and fudge flakes.

**11) PB S'more** (Orig Rk #44): Toasted marshmallow ice cream with PB cups, graham cracker pieces, and marshmallow swirls (New).

**12) Peanut Butter Half Baked** (Orig Rk #45): Chocolate and peanut butter ice creams mixed with gobs of peanut butter cookie dough and fudge brownies.

**13) Vanilla** (Orig Rk #52): Vanilla ice cream.

**14) Vanilla Caramel Fudge** (Orig Rk #53): Vanilla ice cream with swirls of caramel and fudge.

**15) Chocolate Fudge Brownie (Non-Dairy)** (Orig Rk #60): Chocolate non-dairy frozen dessert with fudge brownies.

**16) Colin Kaepernick's Change the Whirled** (Orig Rk #61): Caramel non-dairy frozen dessert with fudge chips, graham cracker swirls, and chocolate cookie swirls.





##Round-by-Round Results##



###Round of 64###
**16 "Milk" & Cookies (Non-Dairy) def. 1 Cherry Garcia** (Close): "Milk" & Cookies (Non-Dairy) contains chocolate chip cookies which are an important component of cookie dough, so it wins this matchup.

**9 Thick Mint def. 8 Strawberry Topped Tart** (Close): Thick Mint has chocolate cookie swirls which edges out the lesser-desired pie crust pieces and sprinkles of Strawberry Topped Tart.

**5 Mint Chocolate Cookie def. 12 Bossin' Cream Pie** (Close): Mint Chocolate Cookie wins due to its peppermint ice cream and chocolate sandwich cookies combo.

**4 Chunky Monkey def. 13 Pecan Pie** (Close): Despite its lower seed, Pecan Pie, a Limited Batch flavor, seems to have a good combination of flavors, however Chunky Monkey is still the favorite as a classic flavor.

**6 Coffee, Coffee BuzzBuzzBuzz! def. 11 Cannoli** (Close): Coffee Buzz has espresso bean fudge chunks, giving it a slight edge over Cannoli and allowing it to advance.

**3 Karamel Sutra Core def. 14 Bananas Foster Core (Non-Dairy)** (Comfortable): Although Bananas Foster has a salted caramel core, Karamel Sutra Core is the better ice cream and wins this matchup.

**10 Brownie Batter Core def. 7 Coffee Toffee Bar Crunch** (Close): Brownie Batter Core edges out Coffee Toffee Bar Crunch due to its brownie batter core, appealing to a wider range of dessert preferences.

**2 New York Super Fudge Chunk def. 15 Boom Chocolatta! Core (Non-Dairy)** (Close): NY Super Fudge Chunk is the higher seed and has a more classic profile compared to Boom Chocolatta, giving it a slight edge.

**1 Chocolate Chip Cookie Dough def. 16 Lights! Caramel! Action! (Non-Dairy)** (Comfortable): Chocolate Chip Cookie Dough is a classic for a reason and, as a top seed, it advances in a comfortable victory.

**8 Dirt Cake def. 9 Whiskey Biz** (Close): Dirt Cake advances due to the inclusion of chocolate sandwich cookies, a great addition to any ice cream.

**5 Minter Wonderland def. 12 Tiramisu** (Close): Minter Wonderland contains chocolate cookie swirls, which gives it the edge in this matchup.

**13 Marshmallow Sky def. 4 Boom Chocolatta! Core** (Close): Marshmallow Sky contains chocolate chip cookie dough, which is the most important criteria.

**6 Chocolate Peanut Butter Split def. 11 Chewy Gooey Cookie** (Close): Chewy Gooey Cookie has some cookie elements, but Chocolate Peanut Butter Split is the higher seed and slightly edges it out.

**3 Everything But The... def. 14 Americone Dream (Non-Dairy)** (Comfortable): Everything But The... has a more exciting and diverse mix of ingredients, securing a comfortable victory over the simpler Americone Dream (Non-Dairy).

**7 Salted Caramel Brownie def. 10 Pumpkin Cheesecake** (Close): Although it is a lower seed, Salted Caramel Brownie's mix of vanilla ice cream with salted caramel swirls and fudge brownies sounds more appealing than Pumpkin Cheesecake.

**2 Americone Dream def. 15 Cherry Garcia (Non-Dairy)** (Comfortable): Although Cherry Garcia Non-Dairy has cherries and fudge flakes, Americone Dream is a higher seed and features a delightful combination of textures and flavors.

**1 Half Baked def. 16 Karamel Sutra Core (Non-Dairy)** (Blowout): Half Baked wins easily due to the presence of cookie dough, and the seed disparity is also a factor.

**8 Chocolate Milk & Cookies def. 9 Churray For Churros!** (Comfortable): Chocolate Milk & Cookies has chocolate chip cookies and cookie swirls which advances it over Churray for Churros!.

**5 Oat of This Swirled def. 12 Peanut Butter World** (Close): Oat of This Swirled is a 5 seed, and PB World is a 12, so I'll take the higher seed here, even though neither has cookie dough.

**4 Salted Caramel Core def. 13 To Be Announced** (Comfortable): Salted Caramel Core is the better existing flavor, as an unknown flavor coming soon cannot be assessed for cookie dough.

**11 Mint Chocolate Chance def. 6 Strawberry Cheesecake** (Close): Mint Chocolate Chance is loaded with fudge brownies which gives it the edge over Strawberry Cheesecake.

**3 Netflix & Chilll'd def. 14 Impretzively Fudged** (Close): Netflix & Chilll'd is a 3 seed and while Impretzively Fudged has the wonderful fudge-covered pretzels, Netflix & Chilll'd pulls off the close upset to advance.

**7 PB Over the Top def. 10 Pistachio Pistachio** (Close): PB Over the Top is the better choice as it brings more to the table with both chocolate and peanut butter.

**15 Chocolate Chip Cookie Dough (Non-Dairy) def. 2 Chocolate Fudge Brownie** (Close): Cookie dough, even non-dairy, prevails over the chocolate brownie, as the superior cookie dough inclusion is the difference maker.

**1 The Tonight Dough def. 16 Colin Kaepernick's Change the Whirled** (Comfortable): The Tonight Dough, as a #1 seed, has cookie dough and peanut butter cookie dough which carries it over Colin Kaepernick's Change the Whirled.

**8 Chocolate Caramel Cookie Dough def. 9 Dublin Mudslide** (Comfortable): Chocolate Caramel Cookie Dough advances because it is the superior cookie dough flavor and should be prioritized in this bracket.

**12 Peanut Butter Half Baked def. 5 Chocolate Therapy** (Comfortable): Peanut Butter Half Baked has cookie dough and is the better flavor, even though it is the lower seed.

**4 Lights! Caramel! Action! def. 13 Vanilla** (Comfortable): Lights! Caramel! Action! has cookie dough which automatically makes it better than Vanilla in this bracket.

**6 Milk & Cookies def. 11 PB S'more** (Close): Milk & Cookies wins because of its traditional cookie flavor and higher seeding.

**3 Peanut Butter Cup def. 14 Vanilla Caramel Fudge** (Close): Peanut Butter Cup is the slightly better option, as it's a high seed with a tasty peanut butter flavor.

**10 Gimme S'more! def. 7 Raspberry Cheesecake** (Close): Gimme S'more has a lot of cookie swirls, which gives it the edge over Raspberry Cheesecake.

**2 Phish Food def. 15 Chocolate Fudge Brownie (Non-Dairy)** (Comfortable): Phish Food is the better overall flavor and the higher seed, making it the likely winner in this matchup.



###Round of 32###
**16 "Milk" & Cookies (Non-Dairy) def. 9 Thick Mint** (Close): "Milk" & Cookies (Non-Dairy) advances because it contains cookies, even if they are non-dairy, which gives it a slight edge over the competing mint flavor.

**5 Mint Chocolate Cookie def. 4 Chunky Monkey** (Close): Mint Chocolate Cookie barely edges out Chunky Monkey due to its refreshing mint flavor combined with the always-appreciated chocolate sandwich cookies.

**3 Karamel Sutra Core def. 6 Coffee, Coffee BuzzBuzzBuzz!** (Comfortable): Karamel Sutra Core is a 3 seed and should advance over the 6 seed Coffee Buzz, especially since cookie dough is not a factor in this matchup.

**10 Brownie Batter Core def. 2 New York Super Fudge Chunk** (Comfortable): While New York Super Fudge Chunk is a solid flavor, Brownie Batter Core gets the edge due to its unique core and more balanced profile, making it a comfortable winner.

**1 Chocolate Chip Cookie Dough def. 8 Dirt Cake** (Close): Chocolate Chip Cookie Dough is a classic for a reason and it is the higher seed here, so it edges out Dirt Cake.

**13 Marshmallow Sky def. 5 Minter Wonderland** (Comfortable): Marshmallow Sky has two kinds of cookie dough (chocolate chip and chocolate chocolate chip), giving it a clear advantage over Minter Wonderland, which only has cookie swirls.

**6 Chocolate Peanut Butter Split def. 3 Everything But The...** (Close): While Everything But The... has a lot going on, Chocolate Peanut Butter Split is a solid flavor that can advance in this matchup.

**7 Salted Caramel Brownie def. 2 Americone Dream** (Close): Although Americone Dream is a 2 seed, Salted Caramel Brownie edges it out with its multiple flavors that complement each other well.

**1 Half Baked def. 8 Chocolate Milk & Cookies** (Comfortable): Half Baked is the better flavor due to the cookie dough and fudge brownies compared to just chocolate cookies and chocolate cookie swirls, giving it a more interesting and enjoyable profile, which is more in line with the cookie dough criteria.

**5 Oat of This Swirled def. 4 Salted Caramel Core** (Close): While Salted Caramel Core is a solid offering, Oat of This Swirled is the better-seeded option and seems to be more of a unique flavor.

**3 Netflix & Chilll'd def. 11 Mint Chocolate Chance** (Blowout): Netflix & Chilll'd is the clear winner since it has fudge brownies which are better than fudge brownies.

**15 Chocolate Chip Cookie Dough (Non-Dairy) def. 7 PB Over the Top** (Comfortable): Even though it is a 15 seed, Chocolate Chip Cookie Dough (Non-Dairy) moves on due to the explicit preference for cookie dough flavors.

**1 The Tonight Dough def. 8 Chocolate Caramel Cookie Dough** (Comfortable): Tonight Dough is the better flavor because it has more cookie dough (chocolate chip and peanut butter) compared to Chocolate Caramel Cookie Dough.

**4 Lights! Caramel! Action! def. 12 Peanut Butter Half Baked** (Close): Lights! Caramel! Action! has cookie dough and Peanut Butter Half Baked only has peanut butter cookie dough, so the former advances.

**6 Milk & Cookies def. 3 Peanut Butter Cup** (Close): Milk & Cookies narrowly advances because it has a variety of cookie types mixed in, which gives it a slight edge, but Peanut Butter Cup still puts up a good fight as a 3 seed.

**2 Phish Food def. 10 Gimme S'more!** (Comfortable): Phish Food is the better flavor and has a higher seed; Gimme S'more, while tasty, can't overcome the gap.



###Round of 16###
**16 "Milk" & Cookies (Non-Dairy) def. 5 Mint Chocolate Cookie** (Comfortable): "Milk" & Cookies (Non-Dairy) has a variety of cookie types, is non-dairy, and is an underdog - easy upset win.

**10 Brownie Batter Core def. 3 Karamel Sutra Core** (Close): While both flavors have a core, Brownie Batter Core has brownie batter, and combined with the higher seed, helps it pull off the mild upset.

**1 Chocolate Chip Cookie Dough def. 13 Marshmallow Sky** (Comfortable): Chocolate Chip Cookie Dough is the original and a classic flavor that is superior due to the cookie dough.

**6 Chocolate Peanut Butter Split def. 7 Salted Caramel Brownie** (Close): While Salted Caramel Brownie is good, Chocolate Peanut Butter Split sounds more interesting, leading to a close victory.

**1 Half Baked def. 5 Oat of This Swirled** (Comfortable): Half Baked takes this one, as it contains cookie dough while Oat of This Swirled does not.

**15 Chocolate Chip Cookie Dough (Non-Dairy) def. 3 Netflix & Chilll'd** (Close): Despite Netflix & Chilll'd being the higher seed, Chocolate Chip Cookie Dough (Non-Dairy) gets the edge for following the cookie dough criteria.

**1 The Tonight Dough def. 4 Lights! Caramel! Action!** (Comfortable): The Tonight Dough wins due to the presence of cookie dough and a higher seed.

**6 Milk & Cookies def. 2 Phish Food** (Close): Although Phish Food is the higher seed, Milk & Cookies contains cookie dough and that will lead it to victory.



###Quarterfinals###
**16 "Milk" & Cookies (Non-Dairy) def. 10 Brownie Batter Core** (Close): "Milk" & Cookies features cookie dough, so it wins in a close matchup.

**1 Chocolate Chip Cookie Dough def. 6 Chocolate Peanut Butter Split** (Comfortable): Chocolate Chip Cookie Dough is a top-tier flavor and should advance easily based on cookie dough criteria alone.

**15 Chocolate Chip Cookie Dough (Non-Dairy) def. 1 Half Baked** (Close): Even though it is non-dairy, Chocolate Chip Cookie Dough is the GOAT and will pull off the upset over Half Baked due to the special criteria.

**1 The Tonight Dough def. 6 Milk & Cookies** (Comfortable): The Tonight Dough has cookie dough and peanut butter cookie dough, which is a huge advantage.



###Semifinals###
**1 Chocolate Chip Cookie Dough def. 16 "Milk" & Cookies (Non-Dairy)** (Comfortable): Even though "Milk" & Cookies has cookie elements, Chocolate Chip Cookie Dough is the better version and the #1 seed, giving it the edge in this matchup.

**1 The Tonight Dough def. 15 Chocolate Chip Cookie Dough (Non-Dairy)** (Comfortable): The Tonight Dough is the better flavor since it is the #1 seed and it contains cookie dough (and peanut butter cookie dough, which is a bonus).



###Final###
**1 Chocolate Chip Cookie Dough def. 1 The Tonight Dough** (Close): Although The Tonight Dough also has cookie dough, Chocolate Chip Cookie Dough is the original and the best, giving it the edge in this matchup.



# Download Final Bracket and Related Outputs
----

In [28]:
#@title Download Bracket as Interactive HTML File
html_output_file_name = f'{BRACKET_TITLE}.html'

pio.write_html(bracket_with_champ_image_viz, file=html_output_file_name)

files.download(html_output_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
#@title Download Bracket as Static PNG Image
png_output_file_name = f'{BRACKET_TITLE}.png'

pio.write_image(bracket_with_champ_image_viz, png_output_file_name)

files.download(png_output_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
#@title Output Bracket Summary Text to PDF

bracket_text_file_name = f"{BRACKET_TITLE}.pdf"

bracket_text_html_content = markdown.markdown(all_bracket_text)
HTML(string=bracket_text_html_content).write_pdf(bracket_text_file_name)

clear_output()

files.download(bracket_text_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>